In [10]:
import torch
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP, IncrementalClassifier
from avalanche.training.strategies import Naive, CWRStar, Replay, GDumb, Cumulative, LwF, GEM, AGEM, EWC, CoPE
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.training.strategies import BaseStrategy
from avalanche.training.plugins import ReplayPlugin, EWCPlugin
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST, SplitCIFAR10, SplitCIFAR100
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics,ExperienceForgetting
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive
from pl_bolts.models.self_supervised import SwAV
from models.pretrained import PretrainedIncrementalClassifier
from plugins.sparse_ewc import SparseEWCPlugin


# check if selected GPU is available or use CPU
cuda=0
if cuda >= 0:
    assert torch.cuda.device_count() > cuda
device = 'cpu' if cuda == -1 else f'cuda:{cuda}'
print(f'Using device: {device}')
lr=1e-3
softmax_temperature=1
epochs=1
minibatch_size=64
ewc_lambda=1
weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar'
encoder = SwAV.load_from_checkpoint(weight_path, strict=True)
model = PretrainedIncrementalClassifier(encoder,
                                        in_features=2048,
                                        initial_out_features=2).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()
# create split scenario
scenario = SplitCIFAR10(n_experiences=5, return_task_id=False, dataset_root='/share/datasets/')

interactive_logger = InteractiveLogger()
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    ExperienceForgetting(),
    loggers=[interactive_logger])

# create strategy
strategy = Naive(model, optimizer, criterion,
                train_epochs=epochs,
               train_mb_size=minibatch_size, eval_mb_size=minibatch_size,
                device=device,
                 plugins=[SparseEWCPlugin(ewc_lambda)],
                 evaluator=eval_plugin)

# train on the selected scenario with the chosen strategy
print('Starting experiment...')
results = []
for train_batch_info in scenario.train_stream:
    print("Start training on experience ", train_batch_info.current_experience)

    strategy.train(train_batch_info, num_workers=4)
    print("End training on experience ", train_batch_info.current_experience)
    print('Computing accuracy on the test set')
    results.append(strategy.eval(scenario.test_stream[:]))
    

Using device: cuda:0
Files already downloaded and verified
Files already downloaded and verified
Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
0it [00:00, ?it/s]

/home/nislah/.vmgr_repo/dev-2021-02-py38/lib/python3.8/site-packages/avalanche/training/plugins/evaluation.py:84: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


100%|██████████| 157/157 [00:02<00:00, 75.31it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.4991
	Loss_MB/train_phase/train_stream/Task000 = 1.0708
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5702
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8125
-- >> End of training phase << --
End training on experience  0
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|████████████| 32/32 [00:00<00:00, 60.30it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 1.1356
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.6880
-- Starting eval on experience 1 (Task 0) from test stream --
100%|████████████| 32/32 [00:00<00:00, 56.28it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 2.7245
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting ev

RuntimeError: The size of tensor a (10) must match the size of tensor b (8) at non-singleton dimension 0